Przygotowanie danych 

In [1]:
import gzip
import numpy as np
import time

In [2]:
IMAGE_SIZE = 28

validation_image = gzip.open("C:\\Users\\rafal\\python\\t10k-images-idx3-ubyte.gz", "r")
validation_labels = gzip.open("C:\\Users\\rafal\\python\\t10k-labels-idx1-ubyte.gz", "r")

train_image = gzip.open("C:\\Users\\rafal\\python\\train-images-idx3-ubyte.gz", "r")
train_labels = gzip.open("C:\\Users\\rafal\\python\\train-labels-idx1-ubyte.gz", "r")

validation_image.read(16)
validation_labels.read(8)
train_image.read(16)
train_labels.read(8)

def is_prime(x):
    if x in [2,3,5,7]:
        return 1
    elif x in [4,6,8]:
        return 0 
    return -1

is_prime = np.vectorize(is_prime)

def generate_matrices(buffer, length):
    M = np.frombuffer(buffer, dtype=np.uint8)
    M = M.reshape(int(M.shape[0]/(length**2)), length**2)

    return M

def generate_map(labels):
    map = np.argwhere(labels == -1)
    return map.T[0]


X = generate_matrices(train_image.read(), IMAGE_SIZE)
y = is_prime(generate_matrices(train_labels.read(), 1))
map = generate_map(y)

X = np.delete(X, map, 0)
y = np.delete(y, map, 0)



v_X = generate_matrices(validation_image.read(), IMAGE_SIZE)
v_y = is_prime(generate_matrices(validation_labels.read(), 1))
map = generate_map(v_y)

v_X = np.delete(v_X, map, 0)
v_y = np.delete(v_y, map, 0)



Model

In [209]:
theta = np.ndarray((IMAGE_SIZE **2, 1))
step_size = 0.8
epochs = 50
momentum = 0.9
reg = 0.0001

def sigma(z):
    return 1 / (1 + np.exp(-z))

def fit(X, y):
    theta = np.zeros((IMAGE_SIZE **2, 1))
    V = np.zeros((IMAGE_SIZE **2, 1))

    start = time.time()
    iteration_start = start
    permutation = np.arange(X.shape[0])
    for epoch in range(1, epochs+1):
        np.random.shuffle(permutation)
        for i in permutation:
            gradient = (y[i] - sigma(np.matmul(theta.T, X[i])))*X[i].reshape(IMAGE_SIZE**2, 1) - 2*reg*theta
            V = momentum * V + (1-momentum) * gradient
            theta = theta + step_size*V

       
        print(f"{epoch}/{epochs} iteracja zakonczona po {time.time() - iteration_start}")
        iteration_start = time.time()
    print(f"trening zakonczony po {time.time() - start}")
    return theta

def predict(theta, X):
    return sigma(theta.T @ X)

def evaluate(theta, X, y):
    correct = 0.0
    total = 0.0 
    
    for i in range(X.shape[0]):
        if np.ceil(predict(theta, X[i])) == y[i] : 
            correct += 1 
        total += 1 
    return correct/total


In [210]:
theta = fit(X, y)

1/50 iteracja zakonczona po 1.7798306941986084
2/50 iteracja zakonczona po 1.7571682929992676
3/50 iteracja zakonczona po 2.084101915359497
4/50 iteracja zakonczona po 1.7804913520812988
5/50 iteracja zakonczona po 1.7918944358825684
6/50 iteracja zakonczona po 2.058729410171509
7/50 iteracja zakonczona po 1.8965981006622314
8/50 iteracja zakonczona po 1.818906307220459
9/50 iteracja zakonczona po 1.6695103645324707
10/50 iteracja zakonczona po 1.787813663482666
11/50 iteracja zakonczona po 1.8574237823486328
12/50 iteracja zakonczona po 1.8173696994781494
13/50 iteracja zakonczona po 1.8836746215820312
14/50 iteracja zakonczona po 1.8704745769500732
15/50 iteracja zakonczona po 1.9108185768127441
16/50 iteracja zakonczona po 1.8433239459991455
17/50 iteracja zakonczona po 1.852555513381958
18/50 iteracja zakonczona po 1.8119008541107178
19/50 iteracja zakonczona po 1.7539684772491455
20/50 iteracja zakonczona po 1.8452401161193848
21/50 iteracja zakonczona po 1.865354299545288
22/50 i

In [208]:
evaluate(theta, v_X, v_y)

0.8198080279232112